<a href="https://colab.research.google.com/github/antonysama/exploratory/blob/master/OilModelCalib_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/data_science"



/content/drive/My Drive/data_science


In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.environ import *
!pip install cplex -q


In [ ]:
# read table1 in file data.xlsx Sheet4 from a1:b3 which includes headers and row names
import pandas as pd
table1 = pd.read_excel('data.xlsx', sheet_name='Sheet4', header=0, index_col=0, usecols='A:C', nrows=3)
table2 = pd.read_excel('data.xlsx', sheet_name='Sheet5', header=0, index_col=0, usecols='A:B', nrows=3)
table2['Light']['2023-12-01']

50.0

In [ ]:
# pyomo code that will find values for table1['X']['2023-01-01'] and table1['X']['2023-02-01'] 
# to minimize the difference between table2['Light']['2023-12-01'] and 
# the sum of product of table1['X']['2023-01-01'] and table1['Light']['2023-01-01'] and table1['X']['2023-02-01'] and table1['Light']['2023-02-01']


# Define tables
table1 = {'Light': {'2023-01-01': 50, '2023-02-01': 60}, 'X': {'2023-01-01': None, '2023-02-01': None}}
table2 = {'Light': {'2023-12-01': 100}}

# Create model
model = ConcreteModel()

# Define sets
model.I = Set(initialize=['2023-01-01', '2023-02-01'])

# Define variables
model.X = Var(model.I, within=NonNegativeReals)

# Define objective function
def obj_rule(model):
    return (model.X['2023-01-01']*table1['Light']['2023-01-01'] + model.X['2023-02-01']*table1['Light']['2023-02-01'] - table2['Light']['2023-12-01'])
model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def const_rule(model, i):
    return model.X[i] >= 0
model.constraints = Constraint(model.I, rule=const_rule)

# Solve model
SolverFactory('cplex_direct').solve(model)

# Print solution
print('X[2023-01-01] =', model.X['2023-01-01'].value)
print('X[2023-02-01] =', model.X['2023-02-01'].value)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 24.9 MB/s eta 0:00:00
X[2023-01-01] = 0.0
X[2023-02-01] = 0.0


In [ ]:
# a lower bound constraint on the objective function to make it non-negative:
# so that nothing in the solution will be smaller than 0.1 .
# Define sets
model.I = Set(initialize=['2023-01-01', '2023-02-01'])

# Define variables
model.X = Var(model.I, within=NonNegativeReals)

# Define objective function
def obj_rule(model):
    return model.X['2023-01-01']*table1['Light']['2023-01-01'] + model.X['2023-02-01']*table1['Light']['2023-02-01'] - table2['Light']['2023-12-01']
model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def const_rule(model, i):
    return model.X[i] >= 0.1
model.constraints = Constraint(model.I, rule=const_rule)

# Add lower bound constraint on objective function
model.obj_lower = Constraint(expr=model.obj >= 0)

# Solve model
SolverFactory('cplex_direct').solve(model)

# Print solution
print('X[2023-01-01] =', model.X['2023-01-01'].value)
print('X[2023-02-01] =', model.X['2023-02-01'].value)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


X[2023-01-01] = 0.1
X[2023-02-01] = 1.5833333333333335


In [ ]:
# a general multiplication of X and each value under table1['Light']:
# Define list of dates
date_list = ['2023-01-01', '2023-02-01']

# Define set of dates
model.I = Set(initialize=date_list)

# Define variables
model.X = Var(model.I, within=NonNegativeReals)

# Define parameter for Light values
model.Light_values = Param(model.I, initialize={i: table1['Light'][i] for i in model.I})

# Define objective function
def obj_rule(model):
    return sum(model.X[i] * model.Light_values[i] for i in model.I) - table2['Light']['2023-12-01']

model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def const_rule(model, i):
    return model.X[i] >= 0.1

model.constraints = Constraint(model.I, rule=const_rule)

# Add lower bound constraint on objective function
model.obj_lower = Constraint(expr=model.obj >= 0)

# Solve model
SolverFactory('cplex_direct').solve(model)

# Print solution
for i in model.I:
    print('X[{}] = {}'.format(i, model.X[i].value))


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


X[2023-01-01] = 0.1
X[2023-02-01] = 1.5833333333333335


In [ ]:
#a general multiplication of X and each value under table1['Light']:
# read table1 in file data.xlsx Sheet4 from a1:b3 which includes headers and row names
table1 = pd.read_excel('data.xlsx', sheet_name='Sheet4', header=0, index_col=0, usecols='A:C', nrows=3)
table2 = pd.read_excel('data.xlsx', sheet_name='Sheet5', header=0, index_col=0, usecols='A:B', nrows=3)

# Define set of dates from table1
model.I = Set(initialize=table1.index.tolist())

# Define variables
model.X = Var(model.I, within=NonNegativeReals)

# Define parameter for Light values from table1
model.Light_values = Param(model.I, initialize={i: table1.at[i, 'Light'] for i in model.I})

# Define objective function
def obj_rule(model):
    return sum(model.X[i] * model.Light_values[i] for i in model.I) - table2['Light']['2023-12-01']

model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def const_rule(model, i):
    return model.X[i] >= 0.1

model.constraints = Constraint(model.I, rule=const_rule)

# Add lower bound constraint on objective function
model.obj_lower = Constraint(expr=model.obj >= 0)

# Solve model
SolverFactory('cplex_direct').solve(model)

# Print solution
for i in model.I:
    print('X[{}] = {}'.format(i, model.X[i].value))


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


X[2023-01-01 00:00:00] = 0.1
X[2023-02-01 00:00:00] = 3.753846153846154


In [ ]:
#a general multiplication of X and each value under table1['Light']:
# read table1 in file data.xlsx Sheet4 from a1:b3 which includes headers and row names
table1 = pd.read_excel('data.xlsx', sheet_name='Sheet4', header=0, index_col=0, usecols='A:D', nrows=3)
table2 = pd.read_excel('data.xlsx', sheet_name='Sheet5', header=0, index_col=0, usecols='A:B', nrows=3)

# Define set of dates from table1
model.I = Set(initialize=table1.index.tolist())

# Define variables
model.X = Var(model.I, within=NonNegativeReals)

# Define parameter for Light values from table1
model.Light_values = Param(model.I, initialize={i: table1.at[i, 'Light'] for i in model.I})

# Define objective function
def obj_rule(model):
    return sum(model.X[i] * model.Light_values[i] for i in model.I) - table2['Oil']['2023-12-01']

model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def const_rule(model, i):
    return model.X[i] >= 0.1

model.constraints = Constraint(model.I, rule=const_rule)

# Add lower bound constraint on objective function
model.obj_lower = Constraint(expr=model.obj >= 0)

# Solve model
SolverFactory('cplex_direct').solve(model)

# Print solution
for i in model.I:
    print('X[{}] = {}'.format(i, model.X[i].value))

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


X[2023-01-01 00:00:00] = 0.1
X[2023-02-01 00:00:00] = 3.753846153846154


In [ ]:
# Add Medium column 

table1 = pd.read_excel('data.xlsx', sheet_name='Sheet4', header=0, index_col=0, usecols='A:D', nrows=3)
table2 = pd.read_excel('data.xlsx', sheet_name='Sheet5', header=0, index_col=0, usecols='A:B', nrows=3)

# Define set of dates from table1
model = ConcreteModel()
model.I = Set(initialize=table1.index.tolist())

# Define variables
model.X = Var(model.I, within=NonNegativeReals)

# Define parameters
model.Light_values = Param(model.I, initialize={i: table1.at[i, 'Light'] for i in model.I})
model.Medium_values = Param(model.I, initialize={i: table1.at[i, 'Medium'] for i in model.I})

# Define objective function
def obj_rule(model):
    return sum(model.X[i] * (table1.at[i, 'Light'] + table1.at[i, 'Medium']) for i in model.I) - table2['Oil']['2023-12-01']


model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def const_rule(model, i):
    return model.X[i] >= 0.1

model.constraints = Constraint(model.I, rule=const_rule)

# Add lower bound constraint on objective function
model.obj_lower = Constraint(expr=model.obj >= 0)

# Solve model
SolverFactory('cplex_direct').solve(model)

# Print solution
for i in model.I:
    print('X[{}] = {}'.format(i, model.X[i].value))


X[2023-01-01 00:00:00] = 0.1
X[2023-02-01 00:00:00] = 3.246666666666667


In [ ]:
# Multiple years, constraints: constqnt X during the year and miimum X 
# Read data from Excel sheets
table1 = pd.read_excel('data.xlsx', sheet_name='Sheet4', header=0, index_col=0, usecols='A:D') 
table2 = pd.read_excel('data.xlsx', sheet_name='Sheet5', header=0, index_col=0, usecols='A:B')

# Group table1 by year
years = table1.groupby(table1.index.year)

for year, data in years:

  # Define set of dates for this year
  model = ConcreteModel()
  model.I = Set(initialize=data.index.tolist())

  # Define variables
  model.X = Var(model.I, within=NonNegativeReals)
  model.X_year = Var(within=NonNegativeReals)

  # Define parameters
  model.Light_values = Param(model.I, initialize={i: data.at[i, 'Light'] for i in model.I})
  model.Medium_values = Param(model.I, initialize={i: data.at[i, 'Medium'] for i in model.I})

  # Define objective function
  def obj_rule(model):
      return sum(model.X[i] * (data.at[i, 'Light'] + data.at[i, 'Medium']) for i in model.I) - table2['Oil'][str(year) + '-12-01']

  model.obj = Objective(rule=obj_rule, sense=minimize)

  # Define constraints
  def const_rule(model, i):
      return model.X[i] >= 0.1

  # def const_year_rule(model, i):
  #     return model.X[i] == model.X_year

  model.constraints = Constraint(model.I, rule=const_rule)
  model.constraints_year = Constraint(model.I, rule=const_year_rule)

  # Add lower bound constraint on objective function
  model.obj_lower = Constraint(expr=model.obj >= 0)

  # Solve model
  SolverFactory('cplex_direct').solve(model)

  # Print solution
  print('Year:', year)
  for i in model.I:
      print('X[{}] = {}'.format(i, model.X[i].value))
  print('X_year =', model.X_year.value)

Year: 2023
X[2023-01-01 00:00:00] = 1.7857142857142856
X[2023-02-01 00:00:00] = 1.7857142857142856
X_year = 1.7857142857142856
Year: 2024
X[2024-01-01 00:00:00] = 4.511278195488722
X[2024-02-01 00:00:00] = 4.511278195488723
X_year = 4.511278195488723
